# Steam Library Data Analysis
## Data preparation
### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tabulate import tabulate

In [2]:
df= pd.read_csv('/kaggle/input/game-recommendations-on-steam/games.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50796 entries, 0 to 50795
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          50796 non-null  int64  
 1   title           50796 non-null  object 
 2   date_release    50796 non-null  object 
 3   win             50796 non-null  bool   
 4   mac             50796 non-null  bool   
 5   linux           50796 non-null  bool   
 6   rating          50796 non-null  object 
 7   positive_ratio  50796 non-null  int64  
 8   user_reviews    50796 non-null  int64  
 9   price_final     50796 non-null  float64
 10  price_original  50796 non-null  float64
 11  discount        50796 non-null  float64
 12  steam_deck      50796 non-null  bool   
dtypes: bool(4), float64(3), int64(3), object(3)
memory usage: 3.7+ MB


### Identification and conversion of datatypes

In [3]:
data_types = df.dtypes.value_counts()
data_types_table = pd.concat([data_types], axis = 1, keys = ['Sum of Data Type'])
data_types_table

,Sum of Data Type
bool,4
int64,3
object,3
float64,3


In [4]:
object_select = df.select_dtypes(include = 'object').columns
df[object_select]=df[object_select].astype('string')

df['date_release']= pd.to_datetime(df['date_release'])

Here we converted the 'object' columns to strings and the 'date_release' column to a datetime datatype

In [5]:
data_types = df.dtypes.value_counts()
data_types_table = pd.concat([data_types], axis = 1, keys = ['Sum of Data Type'])
data_types_table

,Sum of Data Type
bool,4
int64,3
float64,3
string,2
datetime64[ns],1


### Null Check

In [6]:
null_check = df.isnull().sum()

missing_data_table = pd.concat([null_check], axis = 1, keys = ['Total Missing Data'])

missing_data_table

,Total Missing Data
app_id,0
title,0
date_release,0
win,0
mac,0
linux,0
rating,0
positive_ratio,0
user_reviews,0
price_final,0


### Duplicate Check

In [7]:
duplicate_check = df.duplicated().sum()
print(f"There are {duplicate_check} duplicate rows within this dataset")

There are 0 duplicate rows within this dataset


## Data Analysis
### Initial investigation
#### Top 20 rated steam games released after 2020

In [8]:
top_games = df[(df['date_release'].dt.year >= 2020) & (df['positive_ratio'] >= 90)].sort_values(['user_reviews','positive_ratio'], ascending= [False, False]).head(20)
top_games[['title','user_reviews','positive_ratio']]

,title,user_reviews,positive_ratio
14366,ELDEN RING,528702,92
13107,Phasmophobia,486466,96
48090,Valheim,356617,95
14676,Baldur's Gate 3,269840,95
15628,Sea of Thieves 2023 Edition,253844,90
15079,Raft,240299,93
13408,Hades,214267,98
15195,Vampire Survivors,197109,98
49823,tModLoader,188684,97
14279,Deep Rock Galactic,187823,97


**Summary**

#### Lowest 20 rated steam games released after 2020

In [9]:
bottom_games = df[(df['date_release'].dt.year >= 2020) & (df['positive_ratio'] <= 40)].sort_values(['user_reviews','positive_ratio'], ascending= [False, True]).head(20)
bottom_games[['title','user_reviews','positive_ratio']]

,title,user_reviews,positive_ratio
14190,Overwatch® 2,181198,9
13401,Battlefield™ 2042,153151,40
37392,Mirror 2: Project X,110981,26
48774,eFootball™ 2024,59071,37
50794,PAYDAY 3,29458,38
3191,War of the Three Kingdoms,21276,15
27239,Call of Duty®: Warzone™ 2.0,19770,34
48536,MOBILE SUIT GUNDAM BATTLE OPERATION 2,15685,20
34045,Cultivation Tales,12299,37
12651,Destiny 2: Lightfall,7343,31


**Summary**


### Descriptive Statistics of software pricing across each operating system (Windows, Mac, Linux)

In [10]:
windows_games = df[(df['win'] == True) & (df['price_original'] != 0)]
mac_games = df[(df['mac'] == True) & (df['price_original'] != 0)]
linux_games = df[(df['linux'] == True) & (df['price_original'] != 0)]

os_stats = {
    'Windows': windows_games['price_original'].agg(['mean','median','min','max']).round(2),
    'Mac': mac_games['price_original'].agg(['mean','median','min','max']).round(2),
    'Linux': linux_games['price_original'].agg(['mean','median','min','max']).round(2)
}

os_stats_df = pd.DataFrame(os_stats)
os_stats_final_df = os_stats_df.transpose()
print(tabulate(os_stats_final_df, headers= 'keys', tablefmt='github'))

|         |   mean |   median |   min |    max |
|---------|--------|----------|-------|--------|
| Windows |  10.94 |     7.99 |   0.5 | 299.99 |
| Mac     |  10.31 |     7.99 |   0.9 | 269.99 |
| Linux   |  10.15 |     7.99 |   0.9 | 199.99 |


In [11]:
win_games_count= windows_games['app_id'].count
mac_games_count= mac_games['app_id'].count
linux_games_count= linux_games['app_id'].count

#os_counts =
#os_counts_df= pd.DataFrame(os_counts)
#os_counts_df


#fig= px.pie(os_counts_df, values= 'Game Count')
#fig.show()

**Summary**


#### Game Rating Distribution 

In [12]:
rating_count = df['rating'].value_counts()
rating_count_df= pd.DataFrame(rating_count).reset_index()
rating_count_df.columns = ['Rating', 'Game Count']

fig= px.bar(rating_count_df, x='Rating', y='Game Count', color='Rating', text_auto= True, title= 'Game Rating Distribution')
fig.show()

**Summary**

In [13]:
df['positive_ratio'].corr(df['price_original'])

-0.016677695592262856